In [28]:
import os
import time
import sys

import numpy as np
import faiss

from multiprocessing.dummy import Pool as ThreadPool

In [3]:
def mmap_fvecs(fname):
    x = np.memmap(fname, dtype='int32', mode='r')
    d = x[0]
    return x.view('float32').reshape(-1, d + 1)[:, 1:]

def mmap_bvecs(fname):
    x = np.memmap(fname, dtype='uint8', mode='r')
    d = x[:4].view('int32')[0]
    return x.reshape(-1, d + 4)[:, 4:]

def ivecs_read(fname):
    a = np.fromfile(fname, dtype='int32')
    d = a[0]
    # Wenqi: Format of ground truth (for 10000 query vectors):
    #   1000(topK), [1000 ids]
    #   1000(topK), [1000 ids]
    #        ...     ...
    #   1000(topK), [1000 ids]
    # 10000 rows in total, 10000 * 1001 elements, 10000 * 1001 * 4 bytes
    return a.reshape(-1, d + 1)[:, 1:].copy()

def fvecs_read(fname):
    return ivecs_read(fname).view('float32')

In [4]:
dbname = 'SIFT1M'

if dbname.startswith('SIFT'):
    # SIFT1M to SIFT1000M
    dbsize = int(dbname[4:-1])
    xb = mmap_bvecs('/mnt/scratch/wenqi/Faiss_experiments/bigann/bigann_base.bvecs')
    xq = mmap_bvecs('/mnt/scratch/wenqi/Faiss_experiments/bigann/bigann_query.bvecs')
    gt = ivecs_read('/mnt/scratch/wenqi/Faiss_experiments/bigann/gnd/idx_%dM.ivecs' % dbsize)

    N_VEC = int(dbsize * 1000 * 1000)

    # trim xb to correct size
    xb = xb[:dbsize * 1000 * 1000]

    # Wenqi: load xq to main memory and reshape
    xq = xq.astype('float32').copy()
    xq = np.array(xq, dtype=np.float32)
    gt = np.array(gt, dtype=np.int32)

    print("Vector shapes:")
    print("Base vector xb: ", xb.shape)
    print("Query vector xq: ", xq.shape)
    print("Ground truth gt: ", gt.shape)
else:
    print('unknown dataset', dbname, file=sys.stderr)
    sys.exit(1)

dim = xb.shape[1] # should be 128
nq = xq.shape[0]

Vector shapes:
Base vector xb:  (1000000, 128)
Query vector xq:  (10000, 128)
Ground truth gt:  (10000, 1000)


In [26]:
def get_trained_index(tmpdir, index_key, xt):
    filename = "%s/%s_%s_trained.index" % (
        tmpdir, dbname, index_key)

    if not os.path.exists(filename):
        index = faiss.index_factory(dim, "IVF32,Flat")

        print("Keeping %d train vectors" % xt.shape[0])
        # make sure the data is actually in RAM and in float
        xt = xt.astype('float32').copy()
        index.verbose = True

        t0 = time.time()
        index.train(xt)
        index.verbose = False
        print("train done in %.3f s" % (time.time() - t0))
        print("storing", filename)
        faiss.write_index(index, filename)
    else:
        print("loading", filename)
        index = faiss.read_index(filename)
    return index

def rate_limited_imap(f, l):
    'a thread pre-processes the next element'
    pool = ThreadPool(1)
    res = None
    for i in l:
        res_next = pool.apply_async(f, (i, ))
        if res:
            yield res.get()
        res = res_next
    yield res.get()

def matrix_slice_iterator(x, bs):
    " iterate over the lines of x in blocks of size bs"
    nb = x.shape[0]
    block_ranges = [(i0, min(nb, i0 + bs))
                    for i0 in range(0, nb, bs)]

    return rate_limited_imap(
        lambda i01: x[i01[0]:i01[1]].astype('float32').copy(),
        block_ranges)


def get_populated_index(tmpdir, index_key, xt):

    filename = "%s/%s_%s_populated.index" % (
        tmpdir, dbname, index_key)

    if not os.path.exists(filename):
        index = get_trained_index(tmpdir, index_key, xt)
        i0 = 0
        t0 = time.time()
        for xs in matrix_slice_iterator(xb, 100000):
            i1 = i0 + xs.shape[0]
            print('\radd %d:%d, %.3f s' % (i0, i1, time.time() - t0), end=' ')
            sys.stdout.flush()
            index.add(xs)
            i0 = i1
        print()
        print("Add done in %.3f s" % (time.time() - t0))
        print("storing", filename)
        faiss.write_index(index, filename)
    else:
        print("loading", filename)
        index = faiss.read_index(filename)
        if save_numpy_index:
            print("Saving index to numpy array...")
            chunk = faiss.serialize_index(index)
            np.save("{}.npy".format(filename), chunk)
            print("Finish saving numpy index")
    return index



In [29]:
index_key = "IVF32,Flat"
tmpdir = '../Faiss_indexes'

n_train = int(1e4)
xt = xb[:n_train]

index = get_populated_index(tmpdir, index_key, xt)

loading ../Faiss_indexes/SIFT1M_IVF32,Flat_trained.index
add 900000:1000000, 8.425 s 
Add done in 9.081 s
storing ../Faiss_indexes/SIFT1M_IVF32,Flat_populated.index


In [32]:
ps = faiss.ParameterSpace()
ps.initialize(index)

topK = 1
parametersets = ['nprobe=1', 'nprobe=2', 'nprobe=4', 'nprobe=8', 'nprobe=16', 'nprobe=32']

for param in parametersets:
    print(param, '\t', end=' ')
    sys.stdout.flush()
    ps.set_index_parameters(index, param)

    t0 = time.time()
    D, I = index.search(xq, topK)
    t1 = time.time()

    n_ok = (I[:, :topK] == gt[:, :1]).sum()
    rank = 1
#     for rank in 1, 10, 100:
    n_ok = (I[:, :rank] == gt[:, :1]).sum()
    print("%.4f" % (n_ok / float(nq)), end=' ')
    print("QPS = {}".format(nq / (t1 - t0)))
    #print("%8.3f  " % ((t1 - t0) * 1000.0 / nq), end=' ms')
    # print("%5.2f" % (ivfpq_stats.n_hamming_pass * 100.0 / ivf_stats.ndis))


nprobe=1 	 0.6939 QPS = 1940.2193917302593
nprobe=2 	 0.8633 QPS = 1061.1150640727649
nprobe=4 	 0.9633 QPS = 559.3481966742291
nprobe=8 	 0.9943 QPS = 275.67645401822614
nprobe=16 	 0.9997 QPS = 135.8841252507223
nprobe=32 	 1.0000 QPS = 67.94364550157253


In [48]:
# ONLY Using the first 100 vectors

N = 100

topK = 1
parametersets = ['nprobe=1', 'nprobe=2', 'nprobe=4', 'nprobe=8', 'nprobe=16', 'nprobe=32']

for param in parametersets:
    print(param, '\t', end=' ')
    sys.stdout.flush()
    ps.set_index_parameters(index, param)

    t0 = time.time()
    D, I = index.search(xq[:N], topK)
    t1 = time.time()

    n_ok = (I[:, :topK] == gt[:N, :1]).sum()
    rank = 1
#     for rank in 1, 10, 100:
    n_ok = (I[:, :rank] == gt[:100][:, :1]).sum()
    print("%.4f" % (n_ok / float(N)), end=' ')
    print("QPS = {}".format(nq / (t1 - t0)))
    #print("%8.3f  " % ((t1 - t0) * 1000.0 / nq), end=' ms')
    # print("%5.2f" % (ivfpq_stats.n_hamming_pass * 100.0 / ivf_stats.ndis))

nprobe=1 	 0.7300 QPS = 181038.67403314917
nprobe=2 	 0.8900 QPS = 96019.70619275348
nprobe=4 	 0.9800 QPS = 49867.95579979122
nprobe=8 	 1.0000 QPS = 26191.956767045238
nprobe=16 	 1.0000 QPS = 13012.844676553106
nprobe=32 	 1.0000 QPS = 6195.428713568305
